In [89]:
import pandas as pd 
import seaborn as sns
import numpy as np
import keras_tuner as kt
import tensorflow as tf
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.metrics import F1Score

In [90]:
train_data = pd.read_csv('titanic/train.csv')
test_data = pd.read_csv('titanic/test.csv')

In [91]:
train_data.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [92]:
test_data.columns

Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [93]:
train_data['People'] = train_data['SibSp'] + train_data['Parch'] + 1
test_data['People'] = test_data['SibSp'] + test_data['Parch'] + 1

In [94]:
train_data['Size'] = train_data['Name'] .apply(lambda x : len(x))
test_data['Size'] = test_data['Name'] .apply(lambda x : len(x))
train_data = train_data.drop(columns=['PassengerId', 'Name', 'SibSp', 'Parch','Ticket', 'Cabin'])
test_data = test_data.drop(columns=['PassengerId', 'Name', 'SibSp', 'Parch','Ticket', 'Cabin'])


In [95]:
train_data.columns

Index(['Survived', 'Pclass', 'Name', 'Sex', 'Age', 'Fare', 'Embarked',
       'People', 'Size'],
      dtype='object')

In [71]:
train_data['Age'] = train_data['Age'].fillna(train_data['Age'].mode()[0])
test_data['Age'] = train_data['Age'].fillna(train_data['Age'].mode()[0])
train_data['Fare'] = train_data['Fare'].fillna(train_data['Fare'].mean())
test_data['Fare'] = train_data['Fare'].fillna(train_data['Fare'].mean())
train_data['Embarked'] = train_data['Embarked'].fillna(train_data['Embarked'].mode()[0])
test_data['Embarked'] = train_data['Embarked'].fillna(train_data['Embarked'].mode()[0])

In [72]:
train_data = pd.get_dummies(train_data, dtype='int')
test_data = pd.get_dummies(test_data, dtype='int')

In [73]:
X_train = train_data.drop(columns=['Survived'])
y_train = train_data['Survived'] 

In [86]:
X_train.columns

Index(['Pclass', 'Age', 'Fare', 'People', 'Sex_female', 'Sex_male',
       'Embarked_C', 'Embarked_Q', 'Embarked_S'],
      dtype='object')

In [11]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(9,)))

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  for i in range(hp.Int('No. of Layers', min_value=1, max_value=7, step=1)):
    model.add(keras.layers.Dense(units=hp.Int('n '+str(i), min_value = 32, max_value=512, step=32), activation='relu'))
  dr = hp.Float('dropout', min_value=0, max_value=0.5, step=0.1)
  model.add(keras.layers.Dropout(dr))
  model.add(keras.layers.Dense(1, activation='sigmoid'))
  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-1, 1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.BinaryCrossentropy(),
                metrics=['accuracy'])

  return model

In [59]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     directory='my_dir',
                     factor = 70,
                     project_name='intro_to_kt')

In [60]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [61]:
tuner.search(X_train, y_train, epochs=35, validation_split=0.2, callbacks=[stop_early])
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('No. of Layers')}, dropout was {best_hps.get('dropout')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")


Trial 110 Complete [00h 00m 07s]
val_accuracy: 0.832402229309082

Best val_accuracy So Far: 0.8603351712226868
Total elapsed time: 00h 03m 48s

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 1, dropout was 0.2 and the optimal learning rate for the optimizer
is 0.01.



In [62]:
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train, y_train, epochs=50, validation_split=0.2)
val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))


Epoch 1/50
23/23 [==============================] - 0s 6ms/step - loss: 1.2316 - accuracy: 0.6278 - val_loss: 0.9785 - val_accuracy: 0.6369
Epoch 2/50
23/23 [==============================] - 0s 3ms/step - loss: 0.9133 - accuracy: 0.6910 - val_loss: 0.4241 - val_accuracy: 0.8268
Epoch 3/50
23/23 [==============================] - 0s 4ms/step - loss: 0.6641 - accuracy: 0.7402 - val_loss: 0.5204 - val_accuracy: 0.7765
Epoch 4/50
23/23 [==============================] - 0s 4ms/step - loss: 0.6124 - accuracy: 0.7360 - val_loss: 0.4575 - val_accuracy: 0.7877
Epoch 5/50
23/23 [==============================] - 0s 5ms/step - loss: 0.5761 - accuracy: 0.7556 - val_loss: 0.3902 - val_accuracy: 0.8101
Epoch 6/50
23/23 [==============================] - 0s 3ms/step - loss: 0.5430 - accuracy: 0.7570 - val_loss: 0.5214 - val_accuracy: 0.7877
Epoch 7/50
23/23 [==============================] - 0s 3ms/step - loss: 0.5265 - accuracy: 0.7711 - val_loss: 0.5281 - val_accuracy: 0.8045
Epoch 8/50
23/23 [==

In [63]:
hypermodel = tuner.hypermodel.build(best_hps)
# Retrain the model
hypermodel.fit(X_train, y_train, epochs=best_epoch, validation_split=0.2)

Epoch 1/17
23/23 [==============================] - 1s 7ms/step - loss: 1.5876 - accuracy: 0.6222 - val_loss: 0.5021 - val_accuracy: 0.7765
Epoch 2/17
23/23 [==============================] - 0s 3ms/step - loss: 0.7062 - accuracy: 0.7121 - val_loss: 0.4236 - val_accuracy: 0.7821
Epoch 3/17
23/23 [==============================] - 0s 5ms/step - loss: 0.6507 - accuracy: 0.7430 - val_loss: 0.3967 - val_accuracy: 0.8268
Epoch 4/17
23/23 [==============================] - 0s 4ms/step - loss: 0.5882 - accuracy: 0.7640 - val_loss: 0.3993 - val_accuracy: 0.8101
Epoch 5/17
23/23 [==============================] - 0s 8ms/step - loss: 0.6305 - accuracy: 0.7486 - val_loss: 0.6687 - val_accuracy: 0.7151
Epoch 6/17
23/23 [==============================] - 0s 6ms/step - loss: 0.7069 - accuracy: 0.7163 - val_loss: 0.5554 - val_accuracy: 0.7654
Epoch 7/17
23/23 [==============================] - 0s 6ms/step - loss: 0.5553 - accuracy: 0.7360 - val_loss: 0.4917 - val_accuracy: 0.7654
Epoch 8/17
23/23 [==

In [84]:
m = F1Score()
y_true = y_train[:178].to_numpy().reshape(178,1)
y_pred = np.round(hypermodel.predict(X_train[:178])).astype('int')
m.update_state(y_true, y_pred)
res = m.result()
res.numpy()

6/6 [==============================] - 0s 1ms/step


array([0.71559626], dtype=float32)

In [74]:
predictions = hypermodel.predict(test_data)

14/14 [==============================] - 0s 734us/step


In [75]:
survived = np.round(predictions,0).astype('int')

In [76]:
t = pd.read_csv('titanic/test.csv')
# print(t["PassengerId"].shape, survived.shape)
answer = pd.DataFrame({"PassengerId" : t["PassengerId"], "Survived":survived.ravel()})
answer.to_csv('titanic/predictions.csv',index=False)

In [ ]:
best_hps.values

{'No. of Layers': 6,
 'n 0': 64,
 'dropout': 0.1,
 'learning_rate': 0.001,
 'n 1': 480,
 'n 2': 128,
 'n 3': 128,
 'n 4': 384,
 'n 5': 448,
 'n 6': 32,
 'n 7': 192,
 'n 8': 96,
 'n 9': 128,
 'n 10': 224,
 'tuner/epochs': 100,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 0,
 'tuner/round': 0}

In [16]:
import tensorflow_decision_forests as tfdf
from tensorflow_decision_forests.keras import pd_dataframe_to_tf_dataset 

In [36]:
train_ds = pd_dataframe_to_tf_dataset(train_data, label='Survived')
# test_ds = test_data.insert(column="Survived",value=[0]*418, loc=9)

ValueError: cannot insert Survived, already exists

In [38]:
test_data

,Pclass,Age,Fare,People,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Survived
0,3,22.0,7.2500,1,0,1,0,0,1,0
1,3,38.0,71.2833,2,1,0,1,0,0,0
2,2,26.0,7.9250,1,0,1,0,0,1,0
3,3,35.0,53.1000,1,0,1,0,0,1,0
4,3,35.0,8.0500,3,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...
413,3,24.0,0.0000,1,0,1,0,0,1,0
414,1,44.0,7.9250,1,1,0,0,0,1,0
415,3,24.0,8.0500,1,0,1,0,0,1,0
416,3,34.0,32.5000,1,0,1,0,0,1,0


In [50]:
# test_ds = pd_dataframe_to_tf_dataset(train_data[:178], label='Survived')
test_ds = pd_dataframe_to_tf_dataset(test_data, label='Survived')

In [30]:
model = tfdf.keras.RandomForestModel()
model.fit(train_ds, validation_split=0.2)

Use /tmp/tmp8emijaf_ as temporary training directory


Reading training dataset...
Training dataset read in 0:00:00.110156. Found 713 examples.
Training model...
Model trained in 0:00:00.085700
Compiling model...


[INFO 24-02-22 06:48:38.1120 UTC kernel.cc:1233] Loading model from path /tmp/tmp8emijaf_/model/ with prefix cce4213c9f9e4ee2
[INFO 24-02-22 06:48:38.1404 UTC decision_forest.cc:660] Model loaded with 300 root(s), 37318 node(s), and 9 input feature(s).
[INFO 24-02-22 06:48:38.1404 UTC abstract_model.cc:1344] Engine "RandomForestOptPred" built
[INFO 24-02-22 06:48:38.1404 UTC kernel.cc:1061] Use fast generic engine


Model compiled.


In [31]:
model.compile(["accuracy"])
test_accuracy = model.evaluate(test_ds, return_dict=True, verbose=0)["accuracy"]
print(f"Test accuracy without hyper-parameter tuning: {test_accuracy:.4f}")

Test accuracy without hyper-parameter tuning: 0.8202


In [32]:
tuner = tfdf.tuner.RandomSearch(num_trials=1100)
tuner.choice("min_examples", [2, 5, 7, 10, 14, 21])
tuner.choice("categorical_algorithm", ["CART", "RANDOM"])
local_search_space = tuner.choice("growing_strategy", ["LOCAL"])
local_search_space.choice("max_depth", [3, 4, 5, 6, 8, 10, 12, 14])
global_search_space = tuner.choice("growing_strategy", ["BEST_FIRST_GLOBAL"], merge=True)
global_search_space.choice("max_num_nodes", [16, 32, 64, 128, 256, 512])
tuner.choice("use_hessian_gain", [True, False])
tuner.choice("shrinkage", [0.02, 0.05, 0.10, 0.15, 0.20, 0.25, 0.30])
tuner.choice("num_candidate_attributes_ratio", [0.2, 0.5, 0.9, 1.0])
tuner.choice("split_axis", ["AXIS_ALIGNED"])
oblique_space = tuner.choice("split_axis", ["SPARSE_OBLIQUE"], merge=True)
oblique_space.choice("sparse_oblique_normalization",
                     ["NONE", "STANDARD_DEVIATION", "MIN_MAX"])
oblique_space.choice("sparse_oblique_weights", ["BINARY", "CONTINUOUS"])
oblique_space.choice("sparse_oblique_num_projections_exponent", [1.0, 1.5, 2.0, 2.5])

In [39]:
tuned_model = tfdf.keras.GradientBoostedTreesModel(tuner=tuner)
tuned_model.fit(train_ds, verbose=2)


Use /tmp/tmp67f8kk_x as temporary training directory
Reading training dataset...
Training tensor examples:
Features: {'Pclass': <tf.Tensor 'data:0' shape=(None,) dtype=int64>, 'Age': <tf.Tensor 'data_1:0' shape=(None,) dtype=float64>, 'Fare': <tf.Tensor 'data_2:0' shape=(None,) dtype=float64>, 'People': <tf.Tensor 'data_3:0' shape=(None,) dtype=int64>, 'Sex_female': <tf.Tensor 'data_4:0' shape=(None,) dtype=int64>, 'Sex_male': <tf.Tensor 'data_5:0' shape=(None,) dtype=int64>, 'Embarked_C': <tf.Tensor 'data_6:0' shape=(None,) dtype=int64>, 'Embarked_Q': <tf.Tensor 'data_7:0' shape=(None,) dtype=int64>, 'Embarked_S': <tf.Tensor 'data_8:0' shape=(None,) dtype=int64>}
Label: Tensor("data_9:0", shape=(None,), dtype=int64)
Weights: None
Normalized tensor features:
 {'Pclass': SemanticTensor(semantic=<Semantic.NUMERICAL: 1>, tensor=<tf.Tensor 'Cast:0' shape=(None,) dtype=float32>), 'Age': SemanticTensor(semantic=<Semantic.NUMERICAL: 1>, tensor=<tf.Tensor 'Cast_1:0' shape=(None,) dtype=float32

[WARNING 24-02-22 06:52:12.5711 UTC gradient_boosted_trees.cc:1886] "goss_alpha" set but "sampling_method" not equal to "GOSS".
[WARNING 24-02-22 06:52:12.5711 UTC gradient_boosted_trees.cc:1897] "goss_beta" set but "sampling_method" not equal to "GOSS".
[WARNING 24-02-22 06:52:12.5711 UTC gradient_boosted_trees.cc:1911] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".
[INFO 24-02-22 06:52:12.7017 UTC kernel.cc:771] Start Yggdrasil model training
[INFO 24-02-22 06:52:12.7017 UTC kernel.cc:772] Collect training examples
[INFO 24-02-22 06:52:12.7017 UTC kernel.cc:785] Dataspec guide:
column_guides {
  column_name_pattern: "^__LABEL$"
  type: CATEGORICAL
  categorial {
    min_vocab_frequency: 0
    max_vocab_count: -1
  }
}
default_column_guide {
  categorial {
    max_vocab_count: 2000
  }
  discretized_numerical {
    maximum_num_bins: 255
  }
}
ignore_columns_without_guides: false
detect_numerical_as_discretized_numerical: false

[INFO 24-02-22 06:52

Model trained in 0:02:22.088756
Compiling model...
Model compiled.


[INFO 24-02-22 06:54:34.7843 UTC decision_forest.cc:660] Model loaded with 82 root(s), 2330 node(s), and 9 input feature(s).
[INFO 24-02-22 06:54:34.7843 UTC abstract_model.cc:1344] Engine "GradientBoostedTreesGeneric" built
[INFO 24-02-22 06:54:34.7844 UTC kernel.cc:1061] Use fast generic engine


In [34]:
tuned_model.compile(["accuracy"])
tuned_test_accuracy = tuned_model.evaluate(test_ds, return_dict=True, verbose=0)["accuracy"]
print(f"Test accuracy with the TF-DF hyper-parameter tuner: {tuned_test_accuracy:.4f}")

Test accuracy with the TF-DF hyper-parameter tuner: 0.8258


In [25]:
tuner2 = tfdf.tuner.RandomSearch(num_trials=500, use_predefined_hps=True)

# Define and train the model.
tuned_model2 = tfdf.keras.GradientBoostedTreesModel(tuner=tuner2)
tuned_model2.fit(train_ds, verbose=2)


Use /tmp/tmpf8og66t6 as temporary training directory
Reading training dataset...


[WARNING 24-02-22 06:45:44.0102 UTC gradient_boosted_trees.cc:1886] "goss_alpha" set but "sampling_method" not equal to "GOSS".
[WARNING 24-02-22 06:45:44.0102 UTC gradient_boosted_trees.cc:1897] "goss_beta" set but "sampling_method" not equal to "GOSS".
[WARNING 24-02-22 06:45:44.0102 UTC gradient_boosted_trees.cc:1911] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".


Training tensor examples:
Features: {'Pclass': <tf.Tensor 'data:0' shape=(None,) dtype=int64>, 'Age': <tf.Tensor 'data_1:0' shape=(None,) dtype=float64>, 'Fare': <tf.Tensor 'data_2:0' shape=(None,) dtype=float64>, 'People': <tf.Tensor 'data_3:0' shape=(None,) dtype=int64>, 'Sex_female': <tf.Tensor 'data_4:0' shape=(None,) dtype=int64>, 'Sex_male': <tf.Tensor 'data_5:0' shape=(None,) dtype=int64>, 'Embarked_C': <tf.Tensor 'data_6:0' shape=(None,) dtype=int64>, 'Embarked_Q': <tf.Tensor 'data_7:0' shape=(None,) dtype=int64>, 'Embarked_S': <tf.Tensor 'data_8:0' shape=(None,) dtype=int64>}
Label: Tensor("data_9:0", shape=(None,), dtype=int64)
Weights: None
Normalized tensor features:
 {'Pclass': SemanticTensor(semantic=<Semantic.NUMERICAL: 1>, tensor=<tf.Tensor 'Cast:0' shape=(None,) dtype=float32>), 'Age': SemanticTensor(semantic=<Semantic.NUMERICAL: 1>, tensor=<tf.Tensor 'Cast_1:0' shape=(None,) dtype=float32>), 'Fare': SemanticTensor(semantic=<Semantic.NUMERICAL: 1>, tensor=<tf.Tensor 'C

[INFO 24-02-22 06:45:45.7465 UTC kernel.cc:771] Start Yggdrasil model training
[INFO 24-02-22 06:45:45.7465 UTC kernel.cc:772] Collect training examples
[INFO 24-02-22 06:45:45.7465 UTC kernel.cc:785] Dataspec guide:
column_guides {
  column_name_pattern: "^__LABEL$"
  type: CATEGORICAL
  categorial {
    min_vocab_frequency: 0
    max_vocab_count: -1
  }
}
default_column_guide {
  categorial {
    max_vocab_count: 2000
  }
  discretized_numerical {
    maximum_num_bins: 255
  }
}
ignore_columns_without_guides: false
detect_numerical_as_discretized_numerical: false

[INFO 24-02-22 06:45:45.7467 UTC kernel.cc:391] Number of batches: 1
[INFO 24-02-22 06:45:45.7467 UTC kernel.cc:392] Number of examples: 713
[INFO 24-02-22 06:45:45.7468 UTC kernel.cc:792] Training dataset:
Number of records: 713
Number of columns: 10

Number of columns by type:
	NUMERICAL: 9 (90%)
	CATEGORICAL: 1 (10%)

Columns:

NUMERICAL: 9 (90%)
	0: "Age" NUMERICAL mean:28.8925 min:0.42 max:80 sd:13.0626
	1: "Embarked_C

Model trained in 0:00:45.245856
Compiling model...
Model compiled.


In [28]:
tuned_model2.compile(["accuracy"])
tuned_test_accuracy2 = tuned_model2.evaluate(test_ds, return_dict=True, verbose=0)["accuracy"]
print(f"Test accuracy with the TF-DF hyper-parameter tuner: {tuned_test_accuracy2:.4f}")


Test accuracy with the TF-DF hyper-parameter tuner: 0.7978


In [54]:
predictions_rfm = tuned_model2.predict(test_ds)

1/1 [==============================] - 0s 24ms/step


In [55]:
t = pd.read_csv('titanic/test.csv')
answer = pd.DataFrame({"PassengerId" : t["PassengerId"], "Survived":np.round(predictions_rfm,0).astype('int').ravel()})
answer.to_csv('titanic/predictions.csv',index=False)

In [56]:
tuned_model.summary()

Model: "gradient_boosted_trees_model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1 (1.00 Byte)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 1 (1.00 Byte)
_________________________________________________________________
Type: "GRADIENT_BOOSTED_TREES"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (9):
	Age
	Embarked_C
	Embarked_Q
	Embarked_S
	Fare
	Pclass
	People
	Sex_female
	Sex_male

No weights

Variable Importance: INV_MEAN_MIN_DEPTH:
    1.        "Age"  0.480781 ################
    2.       "Fare"  0.233748 ##
    3. "Embarked_C"  0.229072 ##
    4. "Embarked_Q"  0.212467 #
    5.     "Pclass"  0.204518 #
    6. "Embarked_S"  0.203581 #
    7. "Sex_female"  0.190493 
    8.   "Sex_male"  0.187266 
    9.     "People"  0.184621 

Variable Importance: NUM_AS_ROOT:
    1.        "Age" 40.000000 ################
    2.       "Fare"  9.000000 ##
    3. "Embarked_C" 